In [1]:
import numpy as np
import pandas as pd
import scipy.sparse as sp

import os
os.environ['XLA_PYTHON_CLIENT_PREALLOCATE'] = 'false'
os.environ['CUDA_VISIBLE_DEVICES'] = '1'

import sys
sys.path.append("../..")

from MOFGCN.model import GModel
from MOFGCN.optimizer import Optimizer
from sklearn.model_selection import KFold
from MOFGCN.Entire_Drug_Cell.sampler import Sampler
from MOFGCN.myutils import roc_auc, translate_result, dir_path

In [2]:
data_dir = '../../processed_data/'

In [3]:
# 加载细胞系-药物矩阵
cell_drug = pd.read_csv(data_dir + "cell_drug_binary.csv", index_col=0, header=0)
cell_drug = np.array(cell_drug, dtype=np.float32)
adj_mat_coo_data = sp.coo_matrix(cell_drug).data

# 加载药物-指纹特征矩阵
drug_feature = pd.read_csv(data_dir + "drug_feature.csv", index_col=0, header=0)
feature_drug = np.array(drug_feature, dtype=np.float32)

# 加载细胞系-基因特征矩阵
gene = pd.read_csv(data_dir + "gene_feature.csv.zip", index_col=0, header=0)
gene = np.array(gene, dtype=np.float32)

# 加载细胞系-cna特征矩阵
cna = pd.read_csv(data_dir + "cna_feature.csv.zip", index_col=0, header=0)
cna = np.array(cna, dtype=np.float32)

# 加载细胞系-mutaion特征矩阵
mutation = pd.read_csv("../../../data/CCLE/cell_mutation.csv", index_col=0, header=0)
mutation = np.array(mutation, dtype=np.float32)

# 加载null_mask
null_mask = np.zeros(cell_drug.shape, dtype=np.float32)

In [4]:
k = 5
kfold = KFold(n_splits=k, shuffle=True, random_state=11)

epochs = []
true_datas = pd.DataFrame()
predict_datas = pd.DataFrame()

for fold in range(k):
    for train_index, test_index in kfold.split(np.arange(adj_mat_coo_data.shape[0])):
        sampler = Sampler(cell_drug, train_index, test_index, null_mask)
        model = GModel(adj_mat=sampler.train_data, gene=gene, cna=cna, mutation=mutation, sigma=2, k=2, iterates=3,
                       feature_drug=feature_drug, n_hid1=192, n_hid2=64, alpha=8.70, device="cuda:0")
        opt = Optimizer(model, sampler.train_data, sampler.test_data, sampler.test_mask, sampler.train_mask,
                        roc_auc, lr=1e-3, epochs=1000, device="cuda:0").to("cuda:0")
        epoch, true_data, predict_data = opt()

        epochs.append(epoch)
        true_datas = true_datas.append(translate_result(true_data))
        predict_datas = predict_datas.append(translate_result(predict_data))

epoch:   0 loss:0.976985 auc:0.4557
epoch:  20 loss:0.315871 auc:0.7840
epoch:  40 loss:0.289440 auc:0.8038
epoch:  60 loss:0.270588 auc:0.8169
epoch:  80 loss:0.238524 auc:0.8323
epoch: 100 loss:0.186081 auc:0.8379
epoch: 120 loss:0.137789 auc:0.8401
epoch: 140 loss:0.106669 auc:0.8482
epoch: 160 loss:0.088312 auc:0.8502
epoch: 180 loss:0.078915 auc:0.8521
epoch: 200 loss:0.073467 auc:0.8515
epoch: 220 loss:0.069983 auc:0.8505
epoch: 240 loss:0.067572 auc:0.8493
epoch: 260 loss:0.065816 auc:0.8489
epoch: 280 loss:0.064478 auc:0.8484
epoch: 300 loss:0.063432 auc:0.8488
epoch: 320 loss:0.062588 auc:0.8492
Fit finished.
epoch:   0 loss:1.031445 auc:0.5333
epoch:  20 loss:0.310081 auc:0.7798
epoch:  40 loss:0.286257 auc:0.8022
epoch:  60 loss:0.256685 auc:0.8147
epoch:  80 loss:0.213591 auc:0.8183
epoch: 100 loss:0.156341 auc:0.8325
epoch: 120 loss:0.117207 auc:0.8475
epoch: 140 loss:0.094432 auc:0.8509
epoch: 160 loss:0.082274 auc:0.8523
epoch: 180 loss:0.075445 auc:0.8520
epoch: 200 los

epoch:  60 loss:0.216956 auc:0.7910
epoch:  80 loss:0.154077 auc:0.8024
epoch: 100 loss:0.111210 auc:0.8163
epoch: 120 loss:0.088471 auc:0.8243
epoch: 140 loss:0.077587 auc:0.8289
epoch: 160 loss:0.071745 auc:0.8316
epoch: 180 loss:0.068191 auc:0.8349
epoch: 200 loss:0.065827 auc:0.8371
epoch: 220 loss:0.064155 auc:0.8388
epoch: 240 loss:0.062910 auc:0.8404
epoch: 260 loss:0.061958 auc:0.8421
epoch: 280 loss:0.061192 auc:0.8433
epoch: 300 loss:0.060657 auc:0.8444
epoch: 320 loss:0.060102 auc:0.8455
epoch: 340 loss:0.059681 auc:0.8461
epoch: 360 loss:0.059342 auc:0.8472
epoch: 380 loss:0.059070 auc:0.8476
epoch: 400 loss:0.058804 auc:0.8485
epoch: 420 loss:0.058593 auc:0.8489
epoch: 440 loss:0.058420 auc:0.8492
epoch: 460 loss:0.058282 auc:0.8498
epoch: 480 loss:0.058121 auc:0.8495
epoch: 500 loss:0.058010 auc:0.8497
epoch: 520 loss:0.057893 auc:0.8500
epoch: 540 loss:0.057790 auc:0.8500
epoch: 560 loss:0.058004 auc:0.8502
epoch: 580 loss:0.057635 auc:0.8504
epoch: 600 loss:0.057563 auc

epoch: 740 loss:0.057488 auc:0.8433
epoch: 760 loss:0.057325 auc:0.8429
epoch: 780 loss:0.057281 auc:0.8424
epoch: 800 loss:0.057242 auc:0.8425
epoch: 820 loss:0.057214 auc:0.8429
epoch: 840 loss:0.057196 auc:0.8425
epoch: 860 loss:0.057157 auc:0.8427
epoch: 880 loss:0.057131 auc:0.8430
epoch: 900 loss:0.057105 auc:0.8429
Fit finished.
epoch:   0 loss:0.809056 auc:0.4976
epoch:  20 loss:0.309860 auc:0.7895
epoch:  40 loss:0.276368 auc:0.8019
epoch:  60 loss:0.238327 auc:0.8205
epoch:  80 loss:0.182528 auc:0.8242
epoch: 100 loss:0.133558 auc:0.8301
epoch: 120 loss:0.103517 auc:0.8373
epoch: 140 loss:0.086988 auc:0.8431
epoch: 160 loss:0.078060 auc:0.8458
epoch: 180 loss:0.072695 auc:0.8473
epoch: 200 loss:0.069235 auc:0.8476
epoch: 220 loss:0.066840 auc:0.8480
epoch: 240 loss:0.065099 auc:0.8488
epoch: 260 loss:0.063782 auc:0.8500
epoch: 280 loss:0.062781 auc:0.8510
epoch: 300 loss:0.061977 auc:0.8519
epoch: 320 loss:0.061360 auc:0.8527
epoch: 340 loss:0.060797 auc:0.8531
epoch: 360 los

epoch: 360 loss:0.060527 auc:0.8714
epoch: 380 loss:0.060181 auc:0.8722
epoch: 400 loss:0.059833 auc:0.8727
epoch: 420 loss:0.059556 auc:0.8735
epoch: 440 loss:0.059330 auc:0.8741
epoch: 460 loss:0.059125 auc:0.8749
epoch: 480 loss:0.058939 auc:0.8752
epoch: 500 loss:0.058782 auc:0.8756
epoch: 520 loss:0.058661 auc:0.8755
epoch: 540 loss:0.058530 auc:0.8762
epoch: 560 loss:0.058415 auc:0.8763
epoch: 580 loss:0.058320 auc:0.8764
epoch: 600 loss:0.058308 auc:0.8764
epoch: 620 loss:0.058150 auc:0.8767
epoch: 640 loss:0.058079 auc:0.8768
epoch: 660 loss:0.058015 auc:0.8769
epoch: 680 loss:0.057957 auc:0.8769
epoch: 700 loss:0.057930 auc:0.8770
epoch: 720 loss:0.057861 auc:0.8770
epoch: 740 loss:0.057817 auc:0.8773
epoch: 760 loss:0.057786 auc:0.8773
epoch: 780 loss:0.057752 auc:0.8776
epoch: 800 loss:0.057705 auc:0.8776
epoch: 820 loss:0.057671 auc:0.8775
epoch: 840 loss:0.057653 auc:0.8775
epoch: 860 loss:0.057622 auc:0.8776
epoch: 880 loss:0.057591 auc:0.8776
epoch: 900 loss:0.057569 auc

epoch: 780 loss:0.057504 auc:0.8437
epoch: 800 loss:0.057467 auc:0.8440
epoch: 820 loss:0.057431 auc:0.8441
epoch: 840 loss:0.057410 auc:0.8440
epoch: 860 loss:0.057528 auc:0.8429
epoch: 880 loss:0.057357 auc:0.8444
epoch: 900 loss:0.057315 auc:0.8442
epoch: 920 loss:0.057290 auc:0.8443
epoch: 940 loss:0.057267 auc:0.8444
epoch: 960 loss:0.057246 auc:0.8444
epoch: 980 loss:0.057226 auc:0.8445
Fit finished.
epoch:   0 loss:0.657376 auc:0.4832
epoch:  20 loss:0.303383 auc:0.7853
epoch:  40 loss:0.267004 auc:0.8025
epoch:  60 loss:0.217758 auc:0.8062
epoch:  80 loss:0.158834 auc:0.8182
epoch: 100 loss:0.115306 auc:0.8352
epoch: 120 loss:0.091574 auc:0.8422
epoch: 140 loss:0.079997 auc:0.8447
epoch: 160 loss:0.073739 auc:0.8463
epoch: 180 loss:0.069862 auc:0.8477
epoch: 200 loss:0.067214 auc:0.8488
epoch: 220 loss:0.065307 auc:0.8496
epoch: 240 loss:0.063888 auc:0.8504
epoch: 260 loss:0.062797 auc:0.8510
epoch: 280 loss:0.061941 auc:0.8526
epoch: 300 loss:0.061256 auc:0.8534
epoch: 320 los

In [5]:
file = open("./result_data/epochs.txt", "w")
file.write(str(epochs))
file.close()

In [6]:
pd.DataFrame(true_datas)
# .to_csv("./result_data/true_data.csv")

,0,1,2,3,4,5,6,7,8,9,...,670,671,672,673,674,675,676,677,678,679
0,1.0,0.0,0.0,0.0,0.0,0.0,1.0,1.0,0.0,0.0,...,1.0,0.0,1.0,0.0,1.0,1.0,1.0,1.0,1.0,1.0
0,0.0,0.0,1.0,0.0,1.0,1.0,1.0,1.0,1.0,0.0,...,0.0,1.0,0.0,0.0,0.0,1.0,1.0,1.0,NaN,NaN
0,1.0,0.0,1.0,1.0,0.0,1.0,1.0,0.0,0.0,1.0,...,1.0,1.0,1.0,1.0,1.0,0.0,1.0,1.0,NaN,NaN
0,0.0,0.0,1.0,0.0,1.0,1.0,1.0,1.0,0.0,0.0,...,1.0,0.0,1.0,1.0,1.0,0.0,0.0,0.0,NaN,NaN
0,1.0,0.0,1.0,0.0,0.0,1.0,0.0,1.0,1.0,0.0,...,0.0,0.0,0.0,1.0,1.0,1.0,0.0,1.0,NaN,NaN
0,1.0,0.0,0.0,1.0,1.0,0.0,1.0,1.0,0.0,1.0,...,1.0,0.0,0.0,1.0,0.0,1.0,1.0,0.0,1.0,0.0
0,1.0,1.0,0.0,1.0,1.0,1.0,1.0,0.0,0.0,0.0,...,1.0,0.0,0.0,0.0,1.0,1.0,1.0,0.0,NaN,NaN
0,0.0,0.0,1.0,0.0,1.0,1.0,0.0,1.0,1.0,1.0,...,1.0,1.0,1.0,0.0,1.0,0.0,1.0,1.0,NaN,NaN
0,1.0,1.0,1.0,1.0,1.0,0.0,0.0,1.0,0.0,1.0,...,0.0,0.0,1.0,1.0,1.0,1.0,0.0,0.0,NaN,NaN
0,1.0,1.0,0.0,0.0,1.0,1.0,1.0,1.0,1.0,1.0,...,0.0,1.0,0.0,1.0,1.0,1.0,0.0,0.0,NaN,NaN


In [7]:
pd.DataFrame(predict_datas)
# .to_csv("./result_data/predict_data.csv")

,0,1,2,3,4,5,6,7,8,9,...,670,671,672,673,674,675,676,677,678,679
0,0.868576,0.058790,0.102507,0.006148,0.020440,0.233612,0.446167,0.678781,0.116445,0.013922,...,0.047174,0.016029,0.282744,0.524301,0.284976,0.538071,0.035671,0.516841,0.536692,0.046288
0,0.228814,0.073990,0.244526,0.149348,0.064702,0.489741,0.671249,0.215580,0.064000,0.113737,...,0.364249,0.108783,0.026323,0.040290,0.044773,0.235536,0.025053,0.025238,NaN,NaN
0,0.219618,0.103770,0.732212,0.049687,0.040056,0.064985,0.214793,0.339141,0.819727,0.438155,...,0.211208,0.154923,0.155788,0.143953,0.048559,0.034628,0.536663,0.157378,NaN,NaN
0,0.007608,0.112735,0.641027,0.052349,0.393713,0.227987,0.189017,0.244714,0.045722,0.107239,...,0.891268,0.036496,0.257422,0.837387,0.401722,0.039183,0.137892,0.211953,NaN,NaN
0,0.030300,0.011118,0.027432,0.039444,0.085727,0.900676,0.112216,0.499647,0.180518,0.464726,...,0.082573,0.737268,0.033897,0.574199,0.396607,0.609987,0.002706,0.119587,NaN,NaN
0,0.756622,0.202754,0.144670,0.488761,0.379245,0.047786,0.136079,0.129412,0.422519,0.534281,...,0.269677,0.019903,0.055437,0.096234,0.116660,0.377277,0.613807,0.013275,0.059797,0.039668
0,0.254385,0.067134,0.022274,0.457728,0.591407,0.216952,0.089999,0.057806,0.125101,0.038620,...,0.086732,0.075087,0.005110,0.042151,0.219905,0.027495,0.038620,0.050109,NaN,NaN
0,0.030807,0.024938,0.229978,0.121395,0.709504,0.047858,0.060915,0.090647,0.319893,0.346473,...,0.184744,0.183717,0.131270,0.451379,0.049250,0.022180,0.518675,0.124436,NaN,NaN
0,0.588920,0.346272,0.220388,0.187970,0.276310,0.121086,0.309939,0.318565,0.592881,0.421125,...,0.465707,0.187767,0.875715,0.243219,0.834015,0.409238,0.338140,0.044547,NaN,NaN
0,0.033238,0.028635,0.102854,0.105223,0.805123,0.333690,0.268511,0.613828,0.542118,0.612854,...,0.084146,0.500742,0.165267,0.254499,0.533985,0.156033,0.085707,0.167900,NaN,NaN
